<a href="https://colab.research.google.com/github/suke777777/JHST/blob/main/%E4%BA%BA%E4%BA%8B%E8%A9%95%E4%BE%A1%E6%94%AF%E6%8F%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 準備段階のセル（セル 1: セットアップ）
このセルは、ノートブックを初めて開いたとき、またはランタイムをリセットしたときにのみ実行します。モデルのダウンロードやパッケージのインストールを行います。


In [ ]:
# 1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2. 必要なライブラリのインストール
# llama-cpp-python はGPU利用のため、まずCUDA対応版をインストールします。
# llama-cpp-pythonをインストールする際、GPU (CUDA)を利用する場合は以下のコマンドを使います。
!pip install -U llama-cpp-python pandas gspread chromadb sentence-transformers
# Googleスプレッドシート操作ライブラリ
!pip install gspread pandas
# RAG (埋め込み、ベクトルストア) 用ライブラリ
!pip install chromadb sentence-transformers

In [ ]:
# 3. gspread認証
# Colabのランタイムが自動でGoogleアカウントの認証を行います。
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
print("gspreadの認証が完了しました。")

gspreadの認証が完了しました。


In [ ]:
# 4. RAG用データベースの構築 (最終修正版)
import pandas as pd
# ChromaDBのpersistエラー回避のため、Clientのインポート方法を変更
from chromadb import PersistentClient, Settings
from sentence_transformers import SentenceTransformer

# ファイルパスの指定
TRAINING_DATA_PATH = '/content/drive/MyDrive/JASDF_API/training_data.csv'
DB_PATH = '/content/drive/MyDrive/JASDF_API/chroma_db' # DB保存先

# RAG用モデルのロード
model_name = 'intfloat/multilingual-e5-large'
rag_model = SentenceTransformer(model_name)

try:
    # 1. データの読み込み (文字コードは cp932 を使用)
    df = pd.read_csv(TRAINING_DATA_PATH, encoding='cp932')

    # 2. 列名のクリーニングと指定
    # 列名の前後の不要な空白を除去
    df.columns = df.columns.str.strip()

    # 必要な列名を定義 (ユーザー様が確認した実際の列名を使用)
    TARGET_COLUMN = '目標（50～100文字）'
    REPORT_COLUMN = '自己申告（50～100文字）'

    if TARGET_COLUMN not in df.columns or REPORT_COLUMN not in df.columns:
        print("重大エラー: 必要な列名が見つかりません。")
        print(f"現在の列名: {df.columns.tolist()}")
        raise ValueError("CSVファイルの列名が '目標（50～100文字）' または '自己申告（50～100文字）' と一致しません。")

    # 3. ChromaDB クライアントの初期化 (PersistentClientを使用し、自動保存を期待)
    # 💡 修正点: ClientをPersistentClientに変更し、persist()を不要とする
    client = PersistentClient(path=DB_PATH, settings=Settings())
    collection_name = "jasdf_management_data"

    # 既存のコレクションがあれば削除して再作成
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(name=collection_name)

    collection = client.get_or_create_collection(name=collection_name)

    # 4. RAGの対象ドキュメントを作成
    # 💡 修正点: 実際の列名を使用
    documents = (df[TARGET_COLUMN] + " " + df[REPORT_COLUMN]).tolist()

    # 5. ベクトル化とDBへの追加
    embeddings = rag_model.encode(documents, convert_to_tensor=True).tolist()
    ids = [f"doc_{i}" for i in range(len(documents))]

    collection.add(
        embeddings=embeddings,
        documents=documents,
        ids=ids
    )

    # client.persist() は不要になりました
    print(f"RAG用データベースが {DB_PATH} に構築されました。ドキュメント数: {len(documents)}")

except Exception as e:
    print(f"RAG DB構築中に予期せぬエラーが発生しました: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RAG用データベースが /content/drive/MyDrive/JASDF_API/chroma_db に構築されました。ドキュメント数: 293


#2. 実行段階のセル（セル 2: 推論）
このセルは、推論が必要な時に何度でも実行します。Apps Scriptからトリガーされるのもこのセルが実行されることを意味します。

In [ ]:
## 2A. モデルロードと常駐 (認証コード統合版)

from llama_cpp import Llama
from chromadb import PersistentClient, Settings
from sentence_transformers import SentenceTransformer
import torch
import os
import gspread # gspreadを追加
from google.colab import auth # 認証系を追加
from google.auth import default

# --- 💡 修正点: Gspread認証コードをここに統合 ---
print("Gspread認証を開始します...")
auth.authenticate_user()
creds, _ = default()
global gc
gc = gspread.authorize(creds)
print("Gspread認証が完了しました。")
# -------------------------------------------------

# --- デバイスの自動判定 ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.environ["LAMA_CPP_LOG_LEVEL"] = "error"
os.environ["LAMA_CPP_STDERR"] = "2"
if device == 'cpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

# --- 定数設定 ---
LLM_PATH = '/content/drive/MyDrive/GGUF/llama3-8b-instruct-Q5_K_M.gguf'
DB_PATH = '/content/drive/MyDrive/JASDF_API/chroma_db'
COLLECTION_NAME = "jasdf_management_data"

# LLMのロード (CPUに固定)
print("LLM (llama3-8b-instruct) をCPUにロードしています...")
global LLM_MODEL
LLM_MODEL = Llama(
    model_path=LLM_PATH,
    n_ctx=4096,
    n_gpu_layers=0,
    verbose=False
)

# RAG用モデルのロード (自動判定されたデバイスを使用)
global RAG_MODEL, RAG_CLIENT, RAG_COLLECTION
RAG_MODEL = SentenceTransformer('intfloat/multilingual-e5-large', device=device)
RAG_CLIENT = PersistentClient(path=DB_PATH, settings=Settings())
RAG_COLLECTION = RAG_CLIENT.get_collection(name=COLLECTION_NAME)

print("モデルとDBの初期化が完了しました。")

LLM (llama3-8b-instruct) をCPUにロードしています...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


モデルとDBの初期化が完了しました。以降の実行は高速になります。


In [ ]:
## 2B. コメント生成の実行 (出力完結の最終調整)

from llama_cpp import Llama
from chromadb import PersistentClient, Settings
from sentence_transformers import SentenceTransformer
import gspread
import os
import pandas as pd
import torch # torchライブラリを追加

# --- デバイスの自動判定と設定 (前回の設定を維持) ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"RAG埋め込みモデルの処理デバイス: {device}")

os.environ["LAMA_CPP_LOG_LEVEL"] = "error"
os.environ["LAMA_CPP_STDERR"] = "2"
if device == 'cpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

# --- 定数設定 ---
LLM_PATH = '/content/drive/MyDrive/GGUF/llama3-8b-instruct-Q5_K_M.gguf'
DB_PATH = '/content/drive/MyDrive/JASDF_API/chroma_db'
SPREADSHEET_NAME = '評価コメント作成'
COLLECTION_NAME = "jasdf_management_data"

# --- RAG/LLM モデルの初期化 (前回の常駐ロードを想定) ---
# 💡 LLMモデルはステップ2Aでグローバル定義されている前提ですが、もし分離せずに実行する場合は以下のブロックが必要です。
if 'LLM_MODEL' not in globals():
    print("LLM (llama3-8b-instruct) をCPUにロードしています...")
    LLM_MODEL = Llama(
        model_path=LLM_PATH,
        n_ctx=4096,
        n_gpu_layers=0,
        verbose=False
    )
    RAG_MODEL = SentenceTransformer('intfloat/multilingual-e5-large', device=device)
    RAG_CLIENT = PersistentClient(path=DB_PATH, settings=Settings())
    RAG_COLLECTION = RAG_CLIENT.get_collection(name=COLLECTION_NAME)
    print("モデルとDBの初期化が完了しました。")

# --- メイン処理関数 ---
def generate_boss_comment():
    """スプレッドシートからデータを読み込み、LLMでコメントを生成し、書き戻す"""
    try:
        # gcオブジェクトは前のセルで定義済みである前提
        worksheet = gc.open(SPREADSHEET_NAME).worksheet("Sheet1")

        target_tb = worksheet.acell('B2').value
        report_tb = worksheet.acell('C2').value

        if not target_tb or not report_tb:
            worksheet.update_acell('D2', "エラー: 目標または自己申告が空欄です。")
            print("入力データが不足しています。")
            return

        # 3. RAG: 関連データの検索
        query = f"目標: {target_tb} 自己申告: {report_tb}"
        query_embedding = RAG_MODEL.encode(query, convert_to_tensor=True).tolist()

        results = RAG_COLLECTION.query(
            query_embeddings=[query_embedding],
            n_results=3,
            include=['documents']
        )
        relevant_context = "\n".join(results['documents'][0])

        # 4. プロンプトの作成
        # 💡 修正点: 文末まで完結させる指示を追加
        system_prompt = "あなたは、自衛隊の目標管理における経験豊富な上司です。指導的で、建設的かつ適切な「上司コメント」を**必ず日本語で、文末まで適切に完結させ、最大100文字**で生成してください。過去のデータと新しい入力を参考に、具体的な改善点や評価点を明確にしてください。"

        prompt_template = f"""
        {system_prompt}
        # 過去の目標管理データ (RAG)
        ---
        {relevant_context}
        ---
        # 新しい入力データ
        目標01TB: {target_tb}
        自己申告01TB: {report_tb}

        # 出力
        上司コメント:
        (コメントを100文字前後で完結させる):
        """

        # 5. LLMによるコメントの生成
        # 💡 修正点: max_tokensを300に増やし、文章完結のための余裕を持たせる
        response = LLM_MODEL.create_chat_completion(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template}
            ],
            max_tokens=300,
            temperature=0.7
        )

        # 6. 生成されたテキストの抽出と整形
        generated_text = response['choices'][0]['message']['content'].strip()

        # 不要なノイズの除去
        if generated_text.startswith("上司コメント:"):
            generated_text = generated_text.replace("上司コメント:", "").strip()
        generated_text = generated_text.replace("以下は、上司コメントです。", "").strip()
        generated_text = generated_text.replace("(コメントを100文字前後で完結させる):", "").strip()


        # 💡 修正点: 句点「。」を探し、その直後で切り詰めるロジックを採用
        final_comment = generated_text

        # 100文字を超えている場合は、直前の句点で切る
        if len(final_comment) > 100:
            last_period_index = final_comment[:100].rfind('。')
            if last_period_index != -1 and last_period_index > 50: # 50文字以下で切らないように保護
                final_comment = final_comment[:last_period_index + 1]
            else:
                 # 句点が見つからない場合は、そのまま100文字で切る
                final_comment = final_comment[:100]

        # 句点が文末にない場合や、文章が短い場合はそのまま
        elif not final_comment.endswith('。') and len(final_comment) > 50:
             final_comment += '。' # 短い文章の場合、句点を補完して完結

        # 7. スプレッドシートへの書き込み (D2)
        worksheet.update_acell('D2', final_comment)

        print(f"コメント生成とスプレッドシートへの書き込みが完了しました。")
        print(f"生成コメント: {final_comment}")

    except Exception as e:
        error_message = f"処理中にエラーが発生しました: {e}"
        print(error_message)
        try:
            worksheet.update_acell('D2', f"エラー: {e}")
        except:
            pass

# --- 関数を実行 ---
generate_boss_comment()

RAG埋め込みモデルの処理デバイス: cuda
コメント生成とスプレッドシートへの書き込みが完了しました。
生成コメント: 「目標01TBのDX推進計画立案は、成果が出ているが、初期導入の完了は半分までにとどまっている。計画の具体的な実施計画や人員の確保の具体的なスケジュールが必要である。
